In [8]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

from selenium import webdriver
chromedriver = "chromedriver.exe"
driver = webdriver.Chrome(chromedriver)

import re
import pandas as pd

In [60]:
#make the dataframe that we'll put all the info into
recipeDF = pd.DataFrame(columns = ["Name","Ingredients","Duration","rlink",
                                  "Image URL", "Instructions", "Source"])

In [61]:
categories = ["main-dish", "dessert", "appetizer", "side-dish", "snack",
             "make-ahead", "breakfast", "dinner", "brunch", "lunch", "sub-sandwich",
             "soup"]

for category in categories:
    
    #go through 5 pages for each category
    for x in range(1,6):
        
        #set up each page
        url = f"https://www.chowhound.com/recipes/{category}?page={x}"
        browser.visit(url)
        html = browser.html
        soup = bs(html, 'html.parser')
        
        #start the data collecting        
        
        #finding the links to recipe pages
        recipesGroup = soup.find_all('div', class_ = "fr_bxfoot")
        for rec in recipesGroup:
            #this is the name of the recipe (recName)
            recName = rec.find('h2').text
            
            #this is the link
            link = rec.find('a', href=True)['href']
            
            #visit the recipe page and get the soup for it
            browser.visit(link)
            recipeHtml = browser.html
            recSoup = bs(recipeHtml, 'html.parser')
            
            #get image URL
            if recSoup.find('div', class_ = "fr_hdimgov"):
                imageUrl = recSoup.find('div', class_ = "fr_hdimgov").find('img')['data-src']
            else:
                imageUrl = None
                
            #get duration to make (activeTime)
            if recSoup.find('span', class_ = "frr_active"):
                activeTime = recSoup.find('span', class_ = "frr_active").find('time').text
                activeTime = activeTime.strip()
            else:
                activeTime = None
                
            #get ingredients (ingList)
            ingList = []
            uls = recSoup.find('div', class_ = 'freyja_box81').find_all('ul')
            for ul in uls:
                lis = ul.find_all('li')
                for li in lis:
                    li = re.sub("[\(\[].*?[\)\]]", "", li.text)
                    ingList.append(li)
            
            #get instructions (totInst)
            instGroup = recSoup.find('div', class_ = "fr_instruction_rec").find_all('ol')
            totInst = ""
            for inst in instGroup:
                insts = inst.find_all('li')
                for i in insts:
                    totInst += i.text[2:]
                    
            #put the info into a pandas df
            recipeDF = recipeDF.append({"Name": recName, "Ingredients": ingList, "Duration": activeTime, 
                                        "rlink": link, "Image URL": imageUrl, "Instructions": totInst, 
                                        "Source": "Chowhound"}, ignore_index=True)

WebDriverException: Message: disconnected: received Inspector.detached event
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)


In [63]:
recipeDF.tail()

,Name,Ingredients,Duration,rlink,Image URL,Instructions,Source
1346,Monte Cristo Sandwich,"[1 large egg, 1/3 cup whole milk, Kosher salt,...",None,https://www.chowhound.com/recipes/monte-cristo...,https://search.chow.com/thumbnail/1200/0/www.c...,"Place the egg and milk in a wide, shallow plat...",Chowhound
1347,Pea and Ricotta Bruschetta,"[8 ounces crusty Italian bread, cut crosswise ...",None,https://www.chowhound.com/recipes/pea-and-rico...,https://search.chow.com/thumbnail/1200/0/www.c...,Heat a grill pan or outdoor grill to medium hi...,Chowhound
1348,Sautéed Radicchio and Goat Cheese Bruschetta,"[8 ounces crusty Italian bread, cut crosswise ...",None,https://www.chowhound.com/recipes/sauteed-radi...,https://search.chow.com/thumbnail/1200/0/www.c...,Heat a grill pan or outdoor grill to medium hi...,Chowhound
1349,Apple and Raspberry Fruit Leather,"[1 1/4 cups unsweetened applesauce, 6 ounces f...",None,https://www.chowhound.com/recipes/apple-and-ra...,None,Heat the oven to 170°F and arrange a rack in t...,Chowhound
1350,Chicken Parmesan Sandwich,"[1/2 cup all-purpose flour, seasoned with a pi...",25 min,https://www.chowhound.com/recipes/chicken-parm...,https://search.chow.com/thumbnail/1200/0/www.c...,Position your Baking Steel Griddle on the stov...,Chowhound


In [64]:
recipeDF.to_csv("chowhoundRecipes.csv")

In [65]:
recipeDF.head()

,Name,Ingredients,Duration,rlink,Image URL,Instructions,Source
0,Garlicky Pot Roast,"[8 heads garlic, 1/4 cup olive oil, 3 tablespo...",50 mins,https://www.chowhound.com/recipes/garlicky-pot...,https://search.chow.com/thumbnail/1200/0/www.c...,Cut off the very top (the nonroot end) of the ...,Chowhound
1,Honey-Mustard Glazed Ham,"[1 smoked, bone-in ham, 1 cup honey, 1/4 cup ...",25 mins,https://www.chowhound.com/recipes/honey-mustar...,https://search.chow.com/thumbnail/1200/0/www.c...,Heat the oven to 350°F and arrange a rack in t...,Chowhound
2,Spaghetti and Bone Marrownara Meatballs,"[1 yellow onion, diced , 4 cloves garlic, minc...",40 min,https://www.chowhound.com/recipes/spaghetti-me...,https://search.chow.com/thumbnail/1200/0/www.c...,"In a large pot, mix the onion, garlic, and red...",Chowhound
3,Slow Cooker Pork Ramen,"[3 pounds boneless pork shoulder, cut into 3 e...",30 mins,https://www.chowhound.com/recipes/slow-cooker-...,https://search.chow.com/thumbnail/1200/0/www.c...,Season the pork with salt.This step and the ne...,Chowhound
4,"Slow-Cooked Lamb Shoulder with Orange, Yogurt ...","[4 pound bone-in lamb shoulder roast, Kosher ...",None,https://www.chowhound.com/recipes/slow-cooked-...,https://search.chow.com/thumbnail/1200/0/www.c...,Season the lamb with salt and pepper and allow...,Chowhound
